In [1]:
import geopandas as gpd
import descartes
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import datetime
from collections import defaultdict
import numpy as np
import contextily as ctx

Read in shapefiles:

In [2]:
atx_ct_shp = gpd.read_file('2020_texas_census_tracts.shp')
es_shp = gpd.read_file('20_21_elem_updt_v2.shp')

Ensure coordinate reference systems (CRS) match.

In [3]:
atx_ct_shp.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
es_shp.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

They match, but optionally we can change them to use the same CRS as Open Street Maps:

In [5]:
atx_shp_osm = atx_ct_shp.to_crs({'init': 'EPSG:3857'})
es_shp_osm = es_shp.to_crs({'init': 'EPSG:3857'})

/Users/kpierce/COVID19/austin-spatial-prev/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Calculate the area within each polygon:

In [6]:
es_shp_osm['es_area'] = es_shp_osm.area
atx_shp_osm['cbg_area'] = atx_shp_osm.area

Overlay the two polygons:

In [7]:
es_atx = gpd.overlay(atx_shp_osm, es_shp_osm, how='intersection')

Calculate overlap area and percent overlap:

In [8]:
es_atx['overlap_area'] = es_atx.area
# what percent of an elementary catchment corresponds to the underlying CBG?
es_atx['pct_overlap'] = (es_atx['overlap_area'] / es_atx['cbg_area']) * 100

Grab only the columns of interest to save as a CSV (this will ultimately be further transformed into something like `travel.csv` from the toy contact partitioning model):

In [10]:
overlaps = es_atx[['GEOID', 'ISD', 'CAMPUS', 'SCHL_YEAR', 'pct_overlap']]

Add an age level indicator column:

In [11]:
overlaps['level'] = 'elementary'

/Users/kpierce/COVID19/austin-spatial-prev/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Write to file:

In [12]:
overlaps.to_csv('AISD_elem_census_tract_overlaps.csv')